#Script que recebe o id do arquivo e <span style="background-color: yellow;">desfaz as operações de conciliação bancária</span>

In [ ]:

DECLARE @ROLLBACK BIT= 0;

BEGIN TRANSACTION SCHEDULE;
BEGIN TRY
		/*Region Logical Querys*/

  

    DECLARE @IdArquivoExtrato UNIQUEIDENTIFIER = '7bbd4a4c-cb09-45dc-967a-b047789be61b';  
    DECLARE @VerLancamentosExtratos BIT = 1;
    DECLARE @VerHistoricosDiferentes BIT = 1;
    DECLARE @DeletaArquivo BIT = 1;


    IF ( OBJECT_ID('TEMPDB..#LancamentosDoArquivo') IS NOT NULL )
        DROP TABLE #LancamentosDoArquivo;	

    CREATE TABLE #LancamentosDoArquivo
        (
          Arquivo VARCHAR(MAX) ,
          IdArquivoExtratoLancamento UNIQUEIDENTIFIER ,
          DataLancamento VARCHAR(10) ,
          Valor DECIMAL(18, 2) ,
          Credito VARCHAR(4) ,
          HistoricoArquivo VARCHAR(MAX) ,
          NumeroDocumento VARCHAR(MAX) ,
          IdTransacao VARCHAR(MAX) ,
          HistoricoVinculado VARCHAR(MAX) ,
          DataCadastroLancamento DATETIME
        );
 


    SELECT  'Arquivo==>';
    SELECT  [Chave Gerada] = AE.IdArquivoExtrato ,
            [Data Importação] = CONVERT(VARCHAR(30), AE.DataImportacao, 103) ,
            [Nome Arquivo] = AE.Nome ,
            [Data Inicio] = CONVERT(VARCHAR(30), AE.DataInicio, 103) ,
            [Data Fim] = CONVERT(VARCHAR(30), AE.DataFim, 103) ,
            [Saldo] = AE.Saldo ,
           -- [Conta Bancaria] = CONCAT(CB.Conta, SPACE(1), CB.NomeConta) ,
            [Status] = AE.UltimoHistoricoSituacao ,
            [Total Lançamentos A Debito] = AE.TotalDebito ,
            [Total Lançamentos A Credito] = AE.TotalCredito
    FROM    Despesa.ArquivosExtrato AS AE
          --  JOIN Despesa.ContasBancarias AS CB ON CB.IdContaBancaria = AE.IdContaBancaria
    WHERE   AE.IdArquivoExtrato = '7bbd4a4c-cb09-45dc-967a-b047789be61b'
	


    INSERT  INTO #LancamentosDoArquivo
            SELECT  [Arquivo] = AE.Nome ,
                    AEL.IdArquivoExtratoLancamento ,
                    [Data Lançamento] = CONVERT(VARCHAR(10), AEL.DataLancamento, 103) ,
                    [Valor] = AEL.Valor ,
                    [Lançamento a Credito] = CASE WHEN AEL.Credito = 0
                                                  THEN 'NÂO'
                                                  ELSE 'SIM'
                                             END ,
                    [Historico do Arquivo] = AEL.Historico ,
                    [Numero Documento] = AEL.NumeroDocumento ,
                    [Id Transação] = AEL.FITID ,
                    [Historico Vinculado Base] = HE.Historico ,
                    [Hora do Processamento] = AEL.DataCadastro
            FROM    Despesa.ArquivosExtratoLancamentos AS AEL
                    JOIN Despesa.ArquivosExtrato AS AE ON AE.IdArquivoExtrato = AEL.IdArquivoExtrato
                    JOIN Despesa.HistoricosExtrato AS HE ON HE.IdHistoricoExtrato = AEL.IdHistoricoExtrato
            WHERE   AEL.IdArquivoExtrato = @IdArquivoExtrato
            ORDER BY [Data Lançamento];


    IF @VerLancamentosExtratos = 1
        BEGIN
            SELECT  'Lançamentos====>';
		
            SELECT  LDA.Arquivo ,
                    [Data Lançamento] = LDA.DataLancamento ,
                    [Valor] = LDA.Valor ,
                    [Lançamento a Credito] = LDA.Credito ,
                    [Historico do Arquivo] = LDA.HistoricoArquivo ,
                    [Numero Documento] = LDA.NumeroDocumento ,
                    [Id Transação] = LDA.IdTransacao ,
                    [Historico Vinculado Base] = LDA.HistoricoVinculado ,
                    [Hora do Processamento] = LDA.DataCadastroLancamento
            FROM    #LancamentosDoArquivo AS LDA ORDER BY [Historico do Arquivo],LDA.Valor
			 
       
            IF @VerHistoricosDiferentes = 1
                BEGIN
                    SELECT  'Historicos Diferentes';
                    SELECT  LDA.Arquivo ,
                            [Data Lançamento] = LDA.DataLancamento ,
                            [Valor] = LDA.Valor ,
                            [Lançamento a Credito] = LDA.Credito ,
                            [Historico do Arquivo] = LDA.HistoricoArquivo ,
                            [Numero Documento] = LDA.NumeroDocumento ,
                            [Id Transação] = LDA.IdTransacao ,
                            [Historico Vinculado Base] = LDA.HistoricoVinculado ,
                            [Hora do Processamento] = LDA.DataCadastroLancamento
                    FROM    #LancamentosDoArquivo AS LDA
                    WHERE   [LDA].[HistoricoArquivo] <> [LDA].[HistoricoVinculado];

                END;

        END;


    IF @DeletaArquivo = 1
        BEGIN
	  
            DELETE  CA
            FROM    Despesa.ConciliacoesBancariasArquivosExtratoLancamentos CA
                    JOIN #LancamentosDoArquivo AS LDA ON LDA.IdArquivoExtratoLancamento = CA.IdArquivoExtratoLancamento;

            DELETE  AL
            FROM    Despesa.ArquivosExtratoLancamentos AS AL
            WHERE   AL.IdArquivoExtrato = @IdArquivoExtrato;

            DELETE  FROM Despesa.ArquivosExtrato
            WHERE   IdArquivoExtrato = @IdArquivoExtrato;



        END;



       




	   /*End  Logical Querys*/
    IF @ROLLBACK = 0
        BEGIN
            COMMIT TRANSACTION SCHEDULE;
        END;
    ELSE
        BEGIN
            ROLLBACK TRANSACTION SCHEDULE;
        END;

END TRY
BEGIN CATCH 
    
    IF ( @@TRANCOUNT > 0
         OR @ROLLBACK = 1
       )
        BEGIN
            ROLLBACK TRANSACTION SCHEDULE;
        END;


    ROLLBACK TRANSACTION SCHEDULE;
    DECLARE @ErrorNumber INT = ERROR_NUMBER();
    DECLARE @ErrorLine INT = ERROR_LINE();
    DECLARE @ErrorMessage NVARCHAR(4000) = ERROR_MESSAGE();
    DECLARE @ErrorSeverity INT = ERROR_SEVERITY();
    DECLARE @ErrorState INT = ERROR_STATE();

    PRINT 'Actual error number: ' + CAST(@ErrorNumber AS VARCHAR(MAX));
    PRINT 'Actual line number: ' + CAST(@ErrorLine AS VARCHAR(MAX));
    PRINT '@ErrorMessage: ' + CAST(@ErrorMessage AS VARCHAR(MAX));
    PRINT '@ErrorSeverity: ' + CAST(@ErrorLine AS VARCHAR(MAX));
    PRINT '@ErrorState: ' + CAST(@ErrorLine AS VARCHAR(MAX));
    RAISERROR(@ErrorMessage, @ErrorSeverity, @ErrorState);

    
    PRINT 'Error detected, all changes reversed.';
END CATCH;